In [1]:
from duckduckgo_search import DDGS

In [ ]:
# import os
# from typing import Any, Text, Dict, List
# from duckduckgo_search import DDGS  # pip install duckduckgo-search
# from openai import OpenAI  # pip install openai

# from rasa_sdk import Action, Tracker
# from rasa_sdk.executor import CollectingDispatcher
# from rasa_sdk.events import SlotSet


class ActionGiveCareerAdvice(Action):
    def name(self) -> Text:
        return "action_give_career_advice"

    def run(
        self,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: Dict[Text, Any],
    ) -> List[Dict[Text, Any]]:

        # Get slots
        major = tracker.get_slot("current_major") or "your major"
        interest = tracker.get_slot("career_interest") or "careers"
        year = tracker.get_slot("year_of_study") or "your year"
        gpa = tracker.get_slot("gpa") or "strong performance"  # Optional
        has_internship = tracker.get_slot("has_internship") or False

        # Build search query (contextual + timely)
        query = f"2025 {interest} career advice {major} {year} student {gpa} GPA {'with internship experience' if has_internship else 'entry-level'} site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"
        
        try:
            # Search DuckDuckGo (top 3 results)
            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=3))
            
            # Extract snippets & sources
            snippets = [r["body"] for r in results]
            sources = [r["href"] for r in results]
            sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources)])
            
            # LLM prompt for friendly summary
            client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
            prompt = f"""
            You are a warm, encouraging university career counselor. Summarize these search results into 4-5 bullet points of personalized advice for a {year} {major} student interested in {interest}.
            Keep it upbeat, use simple language, add 1-2 emojis. End with a nudge to build skills or network.
            Focus on jobs, skills, internships, resume tips. Base ONLY on these snippets – no external knowledge.
            
            Snippets:
            {chr(10).join(snippets)}
            """
            
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # Cheap & fast
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=300
            )
            advice_text = response.choices[0].message.content.strip()
            
        except Exception as e:
            # Fallback if search/LLM fails
            advice_text = f"""
            Hey, pulling live info can be tricky sometimes! Here's timeless advice for {interest} in {major}:
            - **Explore roles** like entry-level analyst or developer – demand is booming! 🚀
            - **Build skills** with free Coursera courses in Python or data viz.
            - **Internships**: Check Handshake for summer spots; start applying now.
            - **Resume tip**: Quantify wins, e.g., "Led team project saving 20% time."
            Remember, every expert started as a student like you! 💪
            """
            sources_list = "Fallback: General career resources (e.g., your school's site)."
        
        # Utter advice
        dispatcher.utter_message(
            response="utter_advice_core",
            advice_text=advice_text,
            current_major=major,
            career_interest=interest,
            year_of_study=year,
        )
        dispatcher.utter_message(response="utter_encourage_human")
        dispatcher.utter_message(
            response="utter_cite_sources",
            sources_list=sources_list
        )
        
        return []

In [2]:
# Get slots
major = "your major"
interest = "careers"
year = "your year"
gpa = "strong performance"  # Optional
has_internship = False

# Build search query (contextual + timely)
query = f"2025 {interest} career advice {major} {year} student {gpa} GPA {'with internship experience' if has_internship else 'entry-level'} site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"

try:
    # Search DuckDuckGo (top 3 results)
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=3))
    
    # Extract snippets & sources
    snippets = [r["body"] for r in results]
    sources = [r["href"] for r in results]
    sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources)])
    
    # # LLM prompt for friendly summary
    # client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    # prompt = f"""
    # You are a warm, encouraging university career counselor. Summarize these search results into 4-5 bullet points of personalized advice for a {year} {major} student interested in {interest}.
    # Keep it upbeat, use simple language, add 1-2 emojis. End with a nudge to build skills or network.
    # Focus on jobs, skills, internships, resume tips. Base ONLY on these snippets – no external knowledge.
    
    # Snippets:
    # {chr(10).join(snippets)}
    # """
    
    # response = client.chat.completions.create(
    #     model="gpt-4o-mini",  # Cheap & fast
    #     messages=[{"role": "user", "content": prompt}],
    #     temperature=0.3,
    #     max_tokens=300
    # )
    # advice_text = response.choices[0].message.content.strip()
    
except Exception as e:
    # Fallback if search/LLM fails
    advice_text = f"""
    Hey, pulling live info can be tricky sometimes! Here's timeless advice for {interest} in {major}:
    - **Explore roles** like entry-level analyst or developer – demand is booming! 🚀
    - **Build skills** with free Coursera courses in Python or data viz.
    - **Internships**: Check Handshake for summer spots; start applying now.
    - **Resume tip**: Quantify wins, e.g., "Led team project saving 20% time."
    Remember, every expert started as a student like you! 💪
    """
    sources_list = "Fallback: General career resources (e.g., your school's site)."

# # Utter advice
# dispatcher.utter_message(
#     response="utter_advice_core",
#     advice_text=advice_text,
#     current_major=major,
#     career_interest=interest,
#     year_of_study=year,
# )
# dispatcher.utter_message(response="utter_encourage_human")
# dispatcher.utter_message(
#     response="utter_cite_sources",
#     sources_list=sources_list
# )

/tmp/ipykernel_21474/1277786559.py:13: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


In [5]:
from ddgs import DDGS

# Perform a text search
results = DDGS().text("python programming", max_results=10)


snippets = [r["body"] for r in results]
sources = [r["href"] for r in results]
sources_list = ", ".join([f"[{i+1}] {src}" for i, src in enumerate(sources)])

print(snippets)
print(sources_list)

['Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language. Python 3.0, released in 2008, was a major revision and not completely backward-compatible with earlier versions. Beginning with Python 3.5, capabilities and keywords for typing were added to language, allowing optional static typing. Currently only versions in the 3.x series are supported. Python has gained widespread use in the machine learning community. It is widely taught as an introductory programming language. Since 2003, Python has consistently ranked in the top ten of the most popular programming languages in the TIOBE Programming 

In [6]:
from ddgs import DDGS

# Example variables
interest = "data science"
major = "computer science"
year = "junior"
gpa = "3.8"
has_internship = True

# Build query
query = f"2025 {interest} career advice {major} {year} student {gpa} GPA {'with internship experience' if has_internship else 'entry-level'} site:edu OR site:gov OR site:linkedin.com OR site:indeed.com"

# Perform search
results = DDGS().text(query, max_results=10)
for result in results:
    print(result["body"], result["href"])   

Train For A New Career In IT With A Data Science Traineeship And A Guaranteed Placement. Start Your New Career Today With A Guaranteed Job After Training Or Your Course Fees Back. https://duckduckgo.com/y.js?ad_domain=itcareerswitch.co.uk&ad_provider=bingv7aa&ad_type=txad&click_metadata=UZcNSwZ4ZrYXkhyeh4Rur-oBXOgH0p5tQNn7b1dSpOvkFOsWuRtmwjQl152Lykak9V36RQTG_sYWTFZv3gBbjlKwUQrCHTRn1lZP_lS_GDQEyUUbln4sFX-hjQOqmCGFrXHXLhFnR9qE5FcYSnNEeg.9x4qmsYv6Mq1XNntacbWMA&rut=dceeb07233da1ed2d95ed79f7b2957de579941c4f424589fa6e13ff384d7b776&u3=https://www.bing.com/aclick?ld=e8MwuUYDzl9PotoF1qnly1-DVUCUxs9NcB9xxf6-0FQaCqZkI6uUpTcIz5u0vTDDvaN3Y-sEoJK00Sqk_JJgRLAw7A6PP2nMEs_Q1LvGURBjxkySaDtnYjbpiFVzXjynCYNJ7nitiTJ_p-QOkick_YzTiuhGLLNy_mDeVEjwQkoNEkRCRf8WvMBNChdVbDA40Xa-EuX5nnrpVcjXgB_cKqFhqIEEk&u=aHR0cHMlM2ElMmYlMmZpdGNhcmVlcnN3aXRjaC5jby51ayUyZmxhbmRpbmctYiUyZmRhdGEtc2NpZW5jZSUyZiUzZm1zY2xraWQlM2Q4ZTNiNjA4ODA0NDYxOTI4MWRjNDA5ODFlMWVlZjE5ZCUyNnV0bV9zb3VyY2UlM2RiaW5nJTI2dXRtX21lZGl1bSUzZGNwYyUyNnV0bV9jYW1

In [2]:
# from duckduckgo_ai_chat import DuckDuckGoAIChat
import duckduckgo_ai_chat.app as DAI

DAI.checkquery

# chat = DuckDuckGoAIChat()
# response = chat.ask("What are the latest trends in AI?")
# print(response)

<function duckduckgo_ai_chat.app.checkquery(args)>

In [4]:
import duckduckgo_ai_chat.app as app_module

# List all attributes in the package
print(dir(app_module))

# Additionally, you can check the module's help
help(app_module)


['DEBUG', 'Fore', 'MODELS', 'Queue', 'Style', 'Thread', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'accept_terms_of_service', 'checkquery', 'choose_model', 'datetime', 'fetch_response', 'fetch_vqd', 'get_download_folder', 'init', 'json', 'mainrun', 'os', 'print_banner', 'process_stream', 'random', 'requests', 'reset_stdin', 'save_session', 'sys', 'time']
Help on module duckduckgo_ai_chat.app in duckduckgo_ai_chat:

NAME
    duckduckgo_ai_chat.app

DESCRIPTION
    # /// script
    # dependencies = [
    #   "requests",
    #   "colorama",
    # ]
    # ///

FUNCTIONS
    accept_terms_of_service()

    checkquery(args)

    choose_model()

    fetch_response(chat_url, vqd, vqd_hash_1, model, messages)

    fetch_vqd()

    get_download_folder()

    mainrun(args)
        Inspired by duckduckGO-chat-cli

    print_banner()
        Print a colorful welcome banner

    process_stream(response, output_queue)

    reset_stdin()

 

In [4]:
from transformers import pipeline
import torch

# Load a model for text generation
generator = pipeline('text-generation', model='gpt2')
results = generator("Hello, how are you today?", max_length=50)
print(results)


NameError: name 'torch' is not defined